In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor


In [7]:
training_data = datasets.FashionMNIST(
    root='datasets',
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root='datasets',
    train=False,
    download=True,
    transform=ToTensor()
)


In [9]:
batch_size = 64

train_dataloader = DataLoader(training_data,batch_size=batch_size)
test_dataloader = DataLoader(test_data,batch_size=64)

for X,y in test_dataloader:
    print(f'shape X [N,C,H,W]:{X.shape}')
    print(X.dtype)
    print(f'shape y:{y.shape},{y.dtype}')
    break

shape X [N,C,H,W]:torch.Size([64, 1, 28, 28])
torch.float32
shape y:torch.Size([64]),torch.int64


In [ ]:

class NNK(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )
        
    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
    
model = NNK()
print(model)
loss_fn = nn.CrossEntropyLoss()
opt = torch.optim.SGD(model.parameters(),lr=1e-3)


def train(dataloader=train_dataloader, model=model, loss_fn=loss_fn, opt=opt):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)

        opt.zero_grad()
        loss.backward()
        opt.step()

        if batch % 100 == 0:
            loss, curr = loss.item(), (batch+1)*len(X)
            print(f'loss:{loss},[{curr:>5d}/{size:>5d}]')


def test(dataloader=test_dataloader, model=model, loss_fn=loss_fn):
    """
    Purpose: dataloader= test_dataloader
    """
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(
        f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [ ]:
epochs = 5
for t in range(epochs):
    print(f'epoch {t+1}\n')
    train()
    test()
    
print('done')

In [ ]:
import os
model_path = os.path.join('train', 'model', 'dnn.pth')
torch.save(model.state_dict(),model_path)
print("Saved PyTorch Model")


In [ ]:
del model
model = NNK()
model.load_state_dict(torch.load(model_path))


In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')


In [17]:
from ast import mod
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset

class NNF(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(1, 20),
            nn.ReLU(),
            nn.Linear(20, 20),
            nn.ReLU(),
            nn.Linear(20, 1)
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits
    
    
class FD(Dataset):
    def __init__(self, size=100, fu=lambda X : 12*X ^ 8-25*X ^ 7+234*X ^ 6-11*X ^ 5):
        self.size = size
        self.func = fu
    
    def __len__(self):
        return self.size
    
    def __getitem__(self, index):
        return self.func(index)


dataloader = torch.utils.data.DataLoader(
    FD(), batch_size=64, shuffle=True)
model = NNF()
loss_fn = nn.MSELoss()
opt = torch.optim.Adam(model.parameters(),lr=1)
# opt = torch.optim.SGD(model.parameters(), lr=1e-4)


def train(model=model, loss_fn=loss_fn, opt=opt):
    size = 100
    model.train()
    for X in range(size):
        pred = model(torch.tensor([X] , dtype=torch.float32))
        loss = loss_fn(pred, torch.tensor(
            [12*X ^ 8-25*X ^ 7+234*X ^ 6-11*X ^ 5], dtype=torch.float32))

        opt.zero_grad()
        loss.backward()
        opt.step()


def test(model=model, loss_fn=loss_fn):
    size = 100
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X in range(size):
            pred = model(torch.tensor([X], dtype=torch.float32))
            test_loss += F.mse_loss(pred, torch.tensor(
                [12*X ^ 8-25*X ^ 7+234*X ^ 6-11*X ^ 5], dtype=torch.float32))
    test_loss /= 1000
    print(
        f"Avg loss: {test_loss:>8f} \n")
    return test_loss


In [33]:
opt = torch.optim.Adam(model.parameters(), lr=1e3)
epochs = 70000
for t in range(epochs):
    train()
    if t%10000 == 0:
        pre = test()
print('done')


Avg loss: 543324.437500 



KeyboardInterrupt: 

In [37]:
test()

Avg loss: 586710.187500 



tensor(586710.1875)

In [17]:
model(torch.tensor([0],dtype=torch.float32)).item()

1182.943115234375